Scrap Data From Web Artikel Turnbackhoax and Sabehoaks

In [ ]:
# install scrapy
!pip install Scrapy

In [ ]:
# create files for learning
!scrapy startproject webscrapy

New Scrapy project 'webscrapy', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/webscrapy

You can start your first spider with:
    cd webscrapy
    scrapy genspider example example.com


In [ ]:
# get the current working directory
import os
os.getcwd()

'/content'

In [ ]:
# change working directories
os.chdir('/content/webscrapy/webscrapy/spiders')
os.getcwd()

'/content/webscrapy/webscrapy/spiders'

In [ ]:
# create turnbackhoax.py and save it under the webscrapy/webscrapy/spiders directory
%%writefile -a turnbackhoax.py

import scrapy


class TurnbackhoaxSpider(scrapy.Spider):
    name = 'turnbackhoax'
    allowed_domains = ['turnbackhoax.id']
    #start_urls = ['https://turnbackhoax.id/']

    def start_requests(self):
        # Editing the default headers (user-agent)
        yield scrapy.Request(url='https://turnbackhoax.id/', callback=self.parse,
                       headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})


    def parse(self, response):
        article_container = response.xpath('//div[@id="main-content"]/article/div/header')
        


        for article in article_container:
            article_title=article.xpath('.//h3/a/text()').get()
            article_date=article.xpath('.//div/span/text()').get()
            article_author=article.xpath('.//div/span[2]/a/text()').get()

            yield{
                'title':article_title,
                'date':article_date,
                'author':article_author
            }

        
        next_page_url = response.xpath('//div[contains(@class,"nav-links")]/a[contains(@class,"next page-numbers")]/@href').get()

        
        
        if next_page_url:
                yield response.follow(url=next_page_url, callback=self.parse,
                headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})

Writing turnbackhoax.py


In [ ]:
# create turnbackhoax.py and save it under the webscrapy/webscrapy/spiders directory
%%writefile -a saberhoaks.py

import scrapy


class SaberhoaksSpider(scrapy.Spider):
    name = 'saberhoaks'
    allowed_domains = ['saberhoaks.jabarprov.go.id']
    #start_urls = ['https://saberhoaks.jabarprov.go.id/v2/klarifikasi']

    def start_requests(self):
        # Editing the default headers (user-agent)
        yield scrapy.Request(url='https://saberhoaks.jabarprov.go.id/v2/klarifikasi', callback=self.parse,
                       headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})

    

    def parse(self, response):
        global article_date
        article_container = response.xpath('//div[@id="hasil"]/div[contains(@class,"col-md-12")]')

        for article in article_container:
            article_title=article.xpath('.//a/div/div/h3/text()').get()
            article_date=article.xpath('.//a/div/div/div/text()').get()
            link_author=article.xpath('.//a/@href').get()
           
            yield response.follow(url=link_author, callback=self.parse_author, meta={'article_title':article_title})

        
        next_page_url=response.xpath('//ul[@class="pagination justify-content-center"]/li[8]/a/@href').get()

        if next_page_url:
                yield response.follow(url=next_page_url, callback=self.parse,
                headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})


      
    def parse_author(self, response):
        article_title=response.request.meta['article_title']
        article_author=response.xpath('//div[@id="tabel_Keterangan"]/div[contains(@class,"row")]/div[17]/text()').get()

        yield{
                'title':article_title,
                'date':article_date,
                'author':article_author
            }

Writing saberhoaks.py


In [ ]:
# Crawl Data from turnbackhoax
!scrapy crawl turnbackhoax -o turnbackhoax_all.csv

In [ ]:
# Crawl Data from saberhoaks
!scrapy crawl saberhoaks -o saberhoaks_all.csv

File Hasil Scrapy ada di webscrapy/webscrapy/spiders dan download dulu untuk digunakan sebagai similarity check